<a href="https://colab.research.google.com/github/siddharth3444/detect_window/blob/main/Fine_Tunning_Window_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers evaluate
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [4]:
from datasets import load_dataset
ds = load_dataset("mukesh3444/manual-window-detect", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mukesh3444/manual-window-detect contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mukesh3444/manual-window-detect
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major r

/root/.cache/huggingface/datasets/downloads/extracted/ef0d7cf169be3b04fde9fb3eef8787189d2791fac7e595d53faaff3354c18023


Generating train split: 0 examples [00:00, ? examples/s]

===>7
===>2
===>5
===>3
===>8
===>9
===>10


Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

===>4
===>11


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
ds = ds.train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]
train_ds
test_ds


Dataset({
    features: ['image', 'annotation', 'scene_category'],
    num_rows: 2
})

In [7]:
import json
from huggingface_hub import cached_download, hf_hub_url

repo_id = "huggingface/label-files"
filename = "ade20k-id2label.json"
id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)
print(label2id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:655: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


ade20k-id2label.json:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

{'wall': 0, 'building': 1, 'sky': 2, 'floor': 3, 'tree': 4, 'ceiling': 5, 'road': 6, 'bed ': 7, 'windowpane': 8, 'grass': 9, 'cabinet': 10, 'sidewalk': 11, 'person': 12, 'earth': 13, 'door': 14, 'table': 15, 'mountain': 16, 'plant': 17, 'curtain': 18, 'chair': 19, 'car': 20, 'water': 21, 'painting': 22, 'sofa': 23, 'shelf': 24, 'house': 25, 'sea': 26, 'mirror': 27, 'rug': 28, 'field': 29, 'armchair': 30, 'seat': 31, 'fence': 32, 'desk': 33, 'rock': 34, 'wardrobe': 35, 'lamp': 36, 'bathtub': 37, 'railing': 38, 'cushion': 39, 'base': 40, 'box': 41, 'column': 42, 'signboard': 43, 'chest of drawers': 44, 'counter': 45, 'sand': 46, 'sink': 47, 'skyscraper': 48, 'fireplace': 49, 'refrigerator': 50, 'grandstand': 51, 'path': 52, 'stairs': 53, 'runway': 54, 'case': 55, 'pool table': 56, 'pillow': 57, 'screen door': 58, 'stairway': 59, 'river': 60, 'bridge': 61, 'bookcase': 62, 'blind': 63, 'coffee table': 64, 'toilet': 65, 'flower': 66, 'book': 67, 'hill': 68, 'bench': 69, 'countertop': 70, 's

In [25]:
from transformers import MaskFormerFeatureExtractor

checkpoint = "facebook/maskformer-swin-tiny-ade"
feature_extractor = MaskFormerFeatureExtractor.from_pretrained(checkpoint, reduce_labels=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/maskformer/feature_extraction_maskformer.py:28: FutureWarning: The class MaskFormerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MaskFormerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/maskformer/image_processing_maskformer.py:410: FutureWarning: The `size_divisibility` argument is deprecated and will be removed in v4.27. Please use `size_divisor` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/maskformer/image_processing_maskformer.py:417: FutureWarning: The `max_size` argument is deprecated and will be removed in v4.27. Please use size['longest_edge'] instead.
  warnings.warn(


In [9]:
# from torchvision.transforms import ColorJitter

# jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

# def train_transforms(example_batch):
#     images = [jitter(x) for x in example_batch["image"]]
#     labels = [x for x in example_batch["annotation"]]
#     inputs = feature_extractor(images, labels)
#     return inputs


# def val_transforms(example_batch):
#     images = [x for x in example_batch["image"]]
#     labels = [x for x in example_batch["annotation"]]
#     inputs = feature_extractor(images, labels)
#     return inputs

# train_ds.set_transform(train_transforms)
# test_ds.set_transform(val_transforms)


In [26]:
import evaluate

metric = evaluate.load("mean_iou")

**Evaluate**

In [22]:
import numpy as np
import torch
from torch import nn

#def compute_metrics(eval_pred):
#     with torch.no_grad():
#         logits, labels = eval_pred
#         print("==================================")
#         print(labels)
#         logits_tensor = torch.from_numpy(logits)
#         logits_tensor = nn.functional.interpolate(
#             logits_tensor,
#             size=labels.shape[-2:],
#             mode="bilinear",
#             align_corners=False,
#         ).argmax(dim=1)

#         pred_labels = logits_tensor.detach().cpu().numpy()
#         metrics = metric.compute(
#             predictions=pred_labels,
#             references=labels,
#             num_labels=num_labels,
#             ignore_index=255,
#             reduce_labels=False,
#         )
#         for key, value in metrics.items():
#             if isinstance(value, np.ndarray):
#                 metrics[key] = value.tolist()
#         return metrics

def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred

        print("Shapes - logits:", logits.shape, "labels:", labels.shape)

        # Convert logits to torch tensor and move to CPU
        logits_tensor = torch.from_numpy(logits).cpu()

        # Ensure that the labels tensor has the same shape as the logits tensor
        labels = nn.functional.interpolate(
            labels.unsqueeze(0).float(),
            size=logits_tensor.shape[-2:],
            mode="nearest",
            align_corners=False,
        ).squeeze(0).long()

        print("After interpolation - labels:", labels.shape)

        # Interpolate the logits tensor
        logits_tensor = nn.functional.interpolate(
            logits_tensor.unsqueeze(0),
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).squeeze(0).argmax(dim=1)

        print("After interpolation - logits_tensor:", logits_tensor.shape)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels.cpu().numpy(),
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if isinstance(value, np.ndarray):
                metrics[key] = value.tolist()
        return metrics

In [27]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.36.2', '0.26.1')

In [29]:
!pip install --upgrade transformers

In [32]:
from transformers.data.data_collator import default_data_collator

#data_collator = default_data_collator(feature_extractor)

model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-tiny-ade")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=2,
    save_steps=1000,
    num_train_epochs=3,
    save_total_limit=2,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    ##train_dataset=ds["train"],  # Make sure to replace with your train split
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/datasets/formatting/formatting.py:100: RuntimeWarning: divide by zero encountered in remainder
  return table.fast_gather(key % table.num_rows)


IndexError: list index out of range

In [24]:
from transformers import MaskFormerForInstanceSegmentation, TrainingArguments, Trainer

model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-tiny-ade", id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
    output_dir="model_for_window_detection",
    learning_rate=6e-5,
    num_train_epochs=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

RuntimeError: output with shape [] doesn't match the broadcast shape [1]